In [2]:
import cv2
import numpy as np


In [4]:
# Load the image
image = cv2.imread('score.jpg')
cv2.imshow("Image Score", image)
cv2.waitKey(0)

-1

In [7]:
from imutils.perspective import four_point_transform
from imutils import contours
import numpy as np
import cv2

ANSWER_KEY = {0: 1, 1: 4, 2: 0, 3: 3, 4: 1}

# load the image, convert it to grayscale, blur it slightly, then find edges
image = cv2.imread('score.jpg')

if image is None:
    raise FileNotFoundError("The image could not be loaded.")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
_, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Debug output: show the thresholded image
cv2.imshow("Thresholded Image", thresh)
cv2.waitKey(0)
# Find contours in the thresholded image
cnts, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

if not cnts:
    raise ValueError("No contours found in the image. Please check the preprocessing steps.")
questionCnts = []

# loop over the contours
for c in cnts:
    (x, y, w, h) = cv2.boundingRect(c)
    ar = w / float(h)

    if w >= 20 and h >= 20 and ar >= 0.9 and ar <= 1.1:
        questionCnts.append(c)

# sort the question contours top-to-bottom, then initialize the total number of correct answers
questionCnts = contours.sort_contours(questionCnts, method="top-to-bottom")[0]
correct = 0
# each question has 5 possible answers, to loop over the question in batches of 5
for (q, i) in enumerate(np.arange(0, len(questionCnts), 5)):
    print(q,i)
    # sort the contours for the current question from left to right, then initialize the index of the bubbled answer
    cnts = contours.sort_contours(questionCnts[i:i + 5])[0]
    bubbled = None

    # loop over the sorted contours
    for (j, c) in enumerate(cnts):
        mask = np.zeros(thresh.shape, dtype="uint8")
        cv2.drawContours(mask, [c], -1, 255, -1)
        mask = cv2.bitwise_and(thresh, thresh, mask=mask)
        total = cv2.countNonZero(mask)
        #print(total)
        if bubbled is None or total > bubbled[0]:
            bubbled = (total, j)

    color = (0, 0, 255) #Red
    k = ANSWER_KEY[q]

    if k == bubbled[1]:
        color = (0, 255, 0) #Green
        correct += 1
  
    cv2.drawContours(image, [cnts[k]], -1, color, 3)

# grab the test taker
score = (correct / 5.0) * 100
print("[INFO] score: {:.2f}%".format(score))
cv2.putText(image, "{:.2f}%".format(score), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
cv2.imshow("Original", image)
cv2.waitKey(0)


0 0
1 5
2 10
3 15
4 20
[INFO] score: 80.00%


-1